## Check GPU Availability

In [1]:
!nvidia-smi

Fri Nov 24 10:07:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

## Install required libraries

In [2]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.


## Importing libraries

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [5]:
from huggingface_hub import notebook_login
notebook_login()



## Load custom Mental Health conv dataset

In [38]:
data = load_dataset("vibhorag101/phr_mental_therapy_dataset")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/99086 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 99086
    })
})

In [40]:
print(data["train"][0]['text'])

<s>[INST] <<SYS>>
You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>

I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me. [/INST] Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on? </s><s>[INST] I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been

## Model Training

In [42]:
model_name = "typeof/neural-chat-7b-v3-1-sharded" # sharded Mistral-7b model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use Mistral-7b model with custom code
)

Loading checkpoint shards:   0%|          | 0/291 [00:00<?, ?it/s]

In [43]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

In [44]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

peft_model = get_peft_model(model, peft_config)

In [52]:
output_dir = "./Intelv3-neuralchat-Viborag-MentalAssitant"
per_device_train_batch_size = 8 # reduce batch size by 2x only if out-of-memory error
gradient_accumulation_steps = 8  # increase gradient accumulation steps by  2x only if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 200       # training will happen for 150 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
)

In [56]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/99086 [00:00<?, ? examples/s]

In [57]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [49]:
# authenticate WandB for logging metrics
import wandb
wandb.login()



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [58]:
peft_model.config.use_cache = False
trainer.train()

{'loss': 2.2907, 'learning_rate': 0.00019979028262377118, 'epoch': 0.01}
{'loss': 0.8395, 'learning_rate': 0.00019744105246469263, 'epoch': 0.01}
{'loss': 0.5853, 'learning_rate': 0.00019254212296427044, 'epoch': 0.02}
{'loss': 0.5254, 'learning_rate': 0.00018522168236559695, 'epoch': 0.03}
{'loss': 0.5134, 'learning_rate': 0.00017567128158176953, 'epoch': 0.03}
{'loss': 0.492, 'learning_rate': 0.000164140821963114, 'epoch': 0.04}
{'loss': 0.4846, 'learning_rate': 0.00015093201623287631, 'epoch': 0.05}
{'loss': 0.468, 'learning_rate': 0.00013639049369634876, 'epoch': 0.05}
{'loss': 0.4397, 'learning_rate': 0.00012089675630312754, 'epoch': 0.06}
{'loss': 0.4207, 'learning_rate': 0.00010485622221144484, 'epoch': 0.06}
{'loss': 0.4098, 'learning_rate': 8.868861738047158e-05, 'epoch': 0.07}
{'loss': 0.4112, 'learning_rate': 7.281699277636572e-05, 'epoch': 0.08}
{'loss': 0.4083, 'learning_rate': 5.765665457425102e-05, 'epoch': 0.08}
{'loss': 0.4105, 'learning_rate': 4.360429701490934e-05, '

TrainOutput(global_step=200, training_loss=0.5555560445785522, metrics={'train_runtime': 11207.0046, 'train_samples_per_second': 1.142, 'train_steps_per_second': 0.018, 'train_loss': 0.5555560445785522, 'epoch': 0.13})

In [61]:
trainer.push_to_hub()

'https://huggingface.co/Jaykumaran17/Intelv3-neuralchat-Viborag-MentalAssitant/tree/main/'

## Inference Pipeline

In [6]:
# Loading original model
model_name = "typeof/neural-chat-7b-v3-1-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00291.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

model-00002-of-00291.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

model-00003-of-00291.safetensors:   0%|          | 0.00/8.39M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [9]:
# Loading PEFT model
PEFT_MODEL = "Jaykumaran17/Intelv3-neuralchat-Viborag-MentalAssitant"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/291 [00:00<?, ?it/s]

In [12]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

## Compare responses between Original model and PEFT model

In [14]:
query = "How can I prevent anxiety and depression?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How can I prevent anxiety and depression?
  <ASSISTANT>: 
To reduce or manage anxiety and depression, it's important to maintain good mental health practices such as regular exercise, maintaining healthy sleep patterns, eating well-balanced meals, engaging in activities that bring joy like hobbies or spending time with loved ones, practicing mindfulness techniques (like meditation), seeking professional help if needed, and building strong social support networks. Remember, everyone experiences these emotions differently; what works best might vary from person to person.
-------------------------------------------------
PEFT MODEL RESPONSE:
Answer the following question truthfu

In [16]:
query = "How to take care of mental health?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How to take care of mental health?
  <ASSISTANT>: 
To maintain good mental health, it's important to practice self-care and engage in activities that promote wellbeing. Some ways include:
1. Exercise regularly - Physical activity can help reduce stress levels and improve mood.
2. Get enough sleep - Adequate rest helps your brain function properly and supports overall physical health.
3. Eat healthy foods - Proper nutrition provides essential nutrients needed by both body and mind.
4. Connect with others - Strong social connections contribute positively to our emotional state.
5. Engage in hobbies or interests - Pursuing enjoyable pastimes gives us a sense of accomplishment and

In [19]:
query = "What is the warning sign of depression?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: What is the warning sign of depression?
  <ASSISTANT>: 
Some common signs include persistent feelings of sadness or hopelessness, loss of interest in activities that used to bring joy, changes in sleep patterns (sleeping more/less than usual), appetite disturbances, fatigue and low energy levels, difficulty concentrating, irritability, increased sensitivity to rejection, self-loathing, thoughts about suicide, etc. Please note these can vary from person to person. It would be best if someone experiencing such symptoms could seek professional help to understand their situation better.
-------------------------------------------------
PEFT MODEL RESPONSE:
Answer the following que